<a href="https://colab.research.google.com/github/NishatTasnim06/trial/blob/main/StockPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
from IPython.display import display, HTML, Javascript

# Initialize the portfolio object
portfolio = StockPortfolio()

def display_portfolio_ui():
    """Render the HTML for adding stocks and showing portfolio"""
    html_content = """
    <h2>Stock Portfolio Tracker</h2>
    <h3>Add a Stock to Your Portfolio</h3>
    <form action="javascript:void(0);" id="addStockForm">
        <label for="ticker">Stock Ticker (e.g., AAPL):</label>
        <input type="text" id="ticker" name="ticker" required><br><br>
        <small>Don't know the ticker? You can search for it on sites like <a href="https://finance.yahoo.com" target="_blank">Yahoo Finance</a> or <a href="https://www.google.com/finance" target="_blank">Google Finance</a>.</small><br><br>
        <label for="quantity">Quantity:</label>
        <input type="number" id="quantity" name="quantity" required><br><br>
        <button type="submit">Add Stock</button>
    </form>
    <h3>Your Portfolio</h3>
    <div id="portfolioSection">
        <h4>Your portfolio will be displayed here</h4>
    </div>
    <h3>Portfolio Performance</h3>
    <div id="performanceSection">
        <h4>Your portfolio value will be displayed here</h4>
    </div>
    <script>
        // Function to handle the form submission
        document.getElementById("addStockForm").addEventListener("submit", function(event) {
            event.preventDefault();
            let ticker = document.getElementById("ticker").value.toUpperCase();
            let quantity = parseInt(document.getElementById("quantity").value);

            // Validate inputs
            if (!ticker || quantity <= 0) {
                alert("Please enter valid stock ticker and quantity.");
                return;
            }

            // Send the data to Python backend to add stock
            google.colab.kernel.invokeFunction('add_stock', [ticker, quantity], {});
        });
    </script>
    """
    display(HTML(html_content))

# Initialize portfolio and display the UI
portfolio = StockPortfolio()
display_portfolio_ui()

def add_stock(ticker, quantity):
    """Handle adding stocks and display updated portfolio."""
    response = portfolio.add_stock(ticker, quantity)
    updated_portfolio = portfolio.display_portfolio()
    display(HTML(updated_portfolio))

# Register the callback function to allow interaction from JavaScript
from google.colab.output import register_callback
register_callback('add_stock', add_stock)


In [ ]:
# Add stock via the frontend (JavaScript)
def add_stock(ticker, quantity):
    """Handle adding stocks to the portfolio."""
    response = portfolio.add_stock(ticker, quantity)

    # Display updated portfolio
    updated_portfolio = portfolio.display_portfolio()

    # Send updated portfolio details back to the frontend (HTML)
    display(HTML(updated_portfolio))

# Register the function so it can be called from JavaScript
from google.colab.output import register_callback
register_callback('add_stock', add_stock)


In [ ]:
import yfinance as yf
from IPython.display import display, HTML

class StockPortfolio:
    def __init__(self):
        self.portfolio = {}
        self.total_value = 0

    def add_stock(self, ticker, quantity):
        """Add stock to the portfolio."""
        if ticker in self.portfolio:
            self.portfolio[ticker]['quantity'] += quantity
        else:
            self.portfolio[ticker] = {'quantity': quantity}
        return f"Added {quantity} shares of {ticker}."

    def remove_stock(self, ticker, quantity):
        """Remove stock from the portfolio."""
        if ticker in self.portfolio:
            if self.portfolio[ticker]['quantity'] >= quantity:
                self.portfolio[ticker]['quantity'] -= quantity
                if self.portfolio[ticker]['quantity'] == 0:
                    del self.portfolio[ticker]
                return f"Removed {quantity} shares of {ticker}."
            else:
                return f"Not enough shares of {ticker}."
        else:
            return f"{ticker} not found in portfolio."

    def fetch_stock_price(self, ticker):
        """Fetch stock data for a specific ticker."""
        stock = yf.Ticker(ticker)
        stock_data = stock.history(period="1d")
        current_price = stock_data['Close'].iloc[-1]
        return current_price

    def track_performance(self):
        """Calculate total portfolio value."""
        self.total_value = 0
        portfolio_details = []
        for ticker in self.portfolio:
            stock_quantity = self.portfolio[ticker]['quantity']
            stock_price = self.fetch_stock_price(ticker)
            stock_value = stock_quantity * stock_price
            portfolio_details.append({
                'ticker': ticker,
                'quantity': stock_quantity,
                'price': stock_price,
                'value': stock_value
            })
            self.total_value += stock_value
        return portfolio_details

    def display_portfolio(self):
        """Display current portfolio in HTML format."""
        portfolio_details = self.track_performance()

        portfolio_table = """
        <table class="portfolio-table">
            <thead>
                <tr>
                    <th>Stock Ticker</th>
                    <th>Quantity</th>
                    <th>Price ($)</th>
                    <th>Value ($)</th>
                </tr>
            </thead>
            <tbody>
        """

        for stock in portfolio_details:
            portfolio_table += f"""
                <tr>
                    <td>{stock['ticker']}</td>
                    <td>{stock['quantity']}</td>
                    <td>{stock['price']:.2f}</td>
                    <td>{stock['value']:.2f}</td>
                </tr>
            """

        portfolio_table += f"""
            </tbody>
        </table>
        <h3>Total Portfolio Value: ${self.total_value:.2f}</h3>
        """

        return portfolio_table


# Display the stock portfolio UI with CSS styling
def display_portfolio_ui():
    html_content = """
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f9;
            margin: 0;
            padding: 0;
            display: flex;
            flex-direction: column;
            align-items: center;
            padding-top: 20px;
        }

        h2 {
            color: #333;
            font-size: 2.5rem;
        }

        .form-container {
            background-color: #ffffff;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            padding: 20px;
            border-radius: 8px;
            width: 300px;
            margin-bottom: 30px;
        }

        .form-container h3 {
            text-align: center;
            color: #4CAF50;
        }

        .form-container label {
            font-size: 1rem;
            color: #333;
        }

        .form-container input {
            width: 100%;
            padding: 10px;
            margin: 8px 0;
            border: 1px solid #ccc;
            border-radius: 4px;
        }

        .form-container button {
            width: 100%;
            padding: 10px;
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 4px;
            cursor: pointer;
        }

        .form-container button:hover {
            background-color: #45a049;
        }

        .portfolio-table {
            width: 80%;
            margin: 0 auto;
            border-collapse: collapse;
            margin-bottom: 30px;
        }

        .portfolio-table th, .portfolio-table td {
            padding: 12px;
            text-align: left;
            border-bottom: 1px solid #ddd;
        }

        .portfolio-table th {
            background-color: #4CAF50;
            color: white;
        }

        .portfolio-table tr:hover {
            background-color: #f1f1f1;
        }

        #performanceSection {
            font-size: 1.2rem;
            font-weight: bold;
            text-align: center;
            margin-top: 20px;
        }

        h3 {
            color: #000;
            font-size: 1.5rem;
        }

    </style>

    <div class="form-container">
        <h3>Add a Stock to Your Portfolio</h3>
        <form action="javascript:void(0);" id="addStockForm">
            <label for="ticker">Stock Ticker (e.g., AAPL, Tesla,apple,tesla):</label>
            <input type="text" id="ticker" name="ticker" required><br><br>
            <label for="quantity">Quantity:</label>
            <input type="number" id="quantity" name="quantity" required><br><br>
            <button type="submit">Add Stock</button>
        </form>
    </div>

    <h3>Your Portfolio</h3>
    <div id="portfolioSection">
        <h4>Your portfolio will be displayed here</h4>
    </div>

    <h3>Portfolio Performance</h3>
    <div id="performanceSection">
        <h4>Your portfolio value will be displayed here</h4>
    </div>

    <script>
        document.getElementById("addStockForm").addEventListener("submit", function(event) {
            event.preventDefault();
            let ticker = document.getElementById("ticker").value.toUpperCase();
            let quantity = parseInt(document.getElementById("quantity").value);

            if (!ticker || quantity <= 0) {
                alert("Please enter valid stock ticker and quantity.");
                return;
            }

            google.colab.kernel.invokeFunction('add_stock', [ticker, quantity], {});
        });
    </script>
    """
    display(HTML(html_content))

# Initialize portfolio and display the UI
portfolio = StockPortfolio()
display_portfolio_ui()

def add_stock(ticker, quantity):
    """Handle adding stocks and display updated portfolio."""
    response = portfolio.add_stock(ticker, quantity)
    updated_portfolio = portfolio.display_portfolio()
    display(HTML(updated_portfolio))

# Register the callback function to allow interaction from JavaScript
from google.colab.output import register_callback
register_callback('add_stock', add_stock)


Stock Ticker,Quantity,Price ($),Value ($)
AAPL,10,241.84,2418.40
